<a href="https://colab.research.google.com/github/Tealexkay/Midterm-project/blob/main/Day3_wrangling_data_measure_central_tendency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 3: Wrangling Data and Exploring Measures of Central Tendency

In today’s session, we will build on what we learned about **managing data** (from Day 2) and move on to **data wrangling**. We’ll cover slicing, sorting, grouping, and then dive into **measures of central tendency**—including the **mean**, **median**, **mode**, and **trimmed mean**.

## Quick Review from Previous Class
* a. **Recap of Key Concepts from Day 2**
   - How to import data using `pandas`.
   - The basics of exploring a DataFrame (`df.head()`, `df.tail()`, `df.sample()`, `df.dtypes`).
   - Managing missing values and duplicates.
* b. **Selecting a subset of columns and Creating new columns**
   - Much of your analysis depends on selecting **relevant subsets** of your data.
   - Create new columns based on calculations or conditions.



## 1. Loading Data

Lets import the data from the following url using pandas, save it into a variable called `nfl_suspension_df`:

```python
https://raw.githubusercontent.com/liger1apwm/MAT-301_Applied_Stats_Data_Analysis/refs/heads/main/data/nfl-suspensions-data.csv
```


### 🏈 NFL Suspensions Dataset  

This dataset contains information about **NFL player suspensions**, particularly focusing on the **league’s disciplinary actions**. It is based on the story *The NFL’s Uneven History Of Punishing Domestic Violence*. The dataset includes details on players, suspension reasons, and the number of games missed.

#### 📊 Dataset Overview  
The dataset consists of multiple suspensions, categorized by **personal conduct**, **substance abuse**, **performance-enhancing drugs**, and **in-game violence**.

#### 🔍 Column Descriptions  

| **Column**  | **Definition** |
|------------|--------------|
| `name`     | Player's name (formatted as first initial.last name). |
| `team`     | Team the player was part of at the time of suspension. |
| `games`    | Number of games suspended (one regular season = 16 games). |
| `category` | The reason for suspension (e.g., personal conduct, substance abuse, performance-enhancing drugs, in-game violence). |
| `desc.`    | A brief description of the suspension incident. |
| `year`     | The year the suspension occurred. |
| `source`   | The news source reporting the suspension. |

**lets load the data and take a look a the first 10 rows.**

<details><summary>Answer:</summary>
<code>
import pandas as pd</br>
nfl_suspension_df = pd.read_csv('https://raw.githubusercontent.com/liger1apwm/MAT-301_Applied_Stats_Data_Analysis/refs/heads/main/data/nfl-suspensions-data.csv') </br>
nfl_suspension_df.head(10) </br>
</code>
</details>

Lets just select the columns `name, team, games, category, year` and then find out their datatype:

<details><summary>Answer:</summary>
<code>
nfl_suspension_df = nfl_suspension_df[['name','team','games','category','year']] </br>
nfl_suspension_df.dtypes</br>
</code>
</details>

In **pandas**, the object data type is a catch-all data type for string or mixed data (e.g., text, special characters, or even mixed types in a column). When pandas can’t infer a more specific type (like int, float, or datetime), it assigns the object type.

**What do you think the datatypes should be?**

### pandas .astype() function

To change the datatype of a column we can use the .astype function, for example:

For one column at the time:

```python
# Convert 'Age' to int
df['Age'] = df['Age'].astype(int)
```
Multiple columns at once:

```python
df = df.astype({
    'Age': 'int',
    'Join_Date': 'datetime64[ns]',
    'Salary': 'float'
})
```

Using this example lets change the datatypes for each column in our dataframe except for games (we will see later why we are not doing it now):

In [ ]:
nfl_suspension_df = nfl_suspension_df.astype({
    'name': 'string',
    'team': 'string',
    'category': 'string',
    'year': 'int16'
})

nfl_suspension_df.dtypes

Now that the data types have been adjusted to fit our needs, we can proceed with the next steps in working with our DataFrame.

## 2. Wrangling Data (modifying the dataframe)
### 2.1 More on Slicing DataFrames
We can slice or filter rows and columns in `pandas` in multiple ways:
- **`.loc`**: label-based indexing (i.e., based on row labels or column names).
- **`.iloc`**: integer-based indexing (i.e., based on row/column integer positions).

Using .loc:

```python
# Select rows by index labels and specific columns
# Get rows with index labels 1 to 3 and columns 'Player' and 'Score'
print(df.loc[1:3, ['Player', 'Score']])

# Select a single row by its index label
print(df.loc[2])
```
Using .iloc:

```python
# Select rows by integer positions and specific columns
# Get the first 3 rows and the first 2 columns
print(df.iloc[0:3, 0:2])

# Select a single row by its integer position
print(df.iloc[4])
```

We can also slice using **conditional expressions** (e.g., `df[df['Age'] > 30]`) and combine multiple conditions with **AND** (`&`) or **OR** (`|`).

Lets now take care of the games column datatype, **What is an issue that we may encounter if we try to change the datatype of the games column?**

Lets try to change the type to int and see what happends:



In [ ]:
nfl_suspension_df['games'] = nfl_suspension_df['games'].astype(int)

We see that some columns contain string values that cannot be converted to numbers. Therefore, we need to handle those values appropriately. There are various ways to address this issue, but for now, we will simply remove the rows where players are suspended indefinitely. Let’s first identify the columns that contain the “Indef.” value mentioned in the error:

In [ ]:
nfl_suspension_df[nfl_suspension_df['games'] == "Indef."]

We see that the issue is present in the first 6 rows, so we will filter out these problematic rows by keeping only the values that are not equal to “Indef.” using the **Not Equal Operator (!=)**. We will then assign this new filtered DataFrame back to the original variable to overwrite it.

In [ ]:
nfl_suspension_df = nfl_suspension_df[nfl_suspension_df['games'] != "Indef."]
nfl_suspension_df

Here’s how you can inspect 3 rows somewhere in the middle, for example, in the 150’s range, using .loc based on the index:

In [ ]:
nfl_suspension_df.loc[150:153]

Here’s how you can find a specific value in a DataFrame using .loc, by specifying both the index and the column name.

In [ ]:
nfl_suspension_df.loc[150, 'name']

Let's now convert the `games` column into an integer datatype:

In [ ]:
nfl_suspension_df['games'] = nfl_suspension_df['games'].astype(int)
nfl_suspension_df.dtypes

### 2.2 Sorting Data
Use **`.sort_values()`** to sort rows by one or more columns:

```python
df.sort_values(by='column_name') # Sort by a column in ascending order
df.sort_values(by='column_name', ascending=False) # Sort a column in descending order
df.sort_values(by=['column_name1', 'column_name2']) # Sort by two columns in ascending order

```

You can handle missing values while sorting by specifying `na_position='first'` or `na_position='last'`.

Lets sort our dataframe based on the column `year` in descending order to see the most recent year first, think about what looks different in the new sorted dataframe:

In [ ]:
nfl_suspension_df_sorted = nfl_suspension_df.sort_values(by=['year'] , ascending = False, )
nfl_suspension_df_sorted

Now that the rows are sorted by year, we see that the index for each row is all scrambled. Depending on the situation, if you don’t need to preserve the original row order, you may choose to reset the index to keep it sequential. This can be done as follows:

```python
df = df.reset_index(drop=True) # reset index and drop the old index information
```



for our data will look something like this:

In [ ]:
nfl_suspension_df_sorted = nfl_suspension_df_sorted.reset_index(drop=True)
nfl_suspension_df_sorted

### 2.3 Grouping Data
Pandas provides **`.groupby()`** for grouping rows by certain columns, then applying an **aggregation** (e.g., `sum()`, `mean()`, `count()`).

```python
df.groupby('column_name1')['column_name2'].sum() # Group by 1 column and aggregate by sum
df.groupby(['column_name1', 'column_name2'])['column_name3'].mean() # Group by 2 columns and aggregate by the mean
```

This is very useful when you want to understand data aggregated at different levels. For example, what about if we want to know how many games suspended has been each year?

We can answer this question by grouping by the `year` and applying a `sum()` of the `games`, lets apply this to our dataframe:

In [ ]:
nfl_suspension_df_sorted.groupby('year')['games'].sum()

In [ ]:
nfl_suspension_df_sorted.groupby(['year','category'])['games'].sum()

The **`groupby()`** function in pandas is essential for analyzing and summarizing data based on different groups. It allows you to answer important questions, compare values across categories, and perform both numeric and non-numeric operations.

**Is it only for numerical columns? **
No! The `groupby()` function is not limited to **numeric columns**. While you can use aggregation functions like **`sum()`** and **`mean()`** for numerical data, you can also work with **non-numeric columns** to extract unique values, count occurrences, or apply custom functions.

- **Numeric columns** → Use aggregation functions like `sum()`, `mean()`, `max()`, etc.  
- **Non-numeric columns** → Use functions like `size()`, `unique()`, `apply()`, or custom functions.

## 3. Using the .describe() Function
**`.describe()`** in pandas provides summary statistics for numerical columns by default:

- **count**: number of non-null values.
- **mean**: average value.
- **std**: standard deviation.
- **min, max**: minimum and maximum values.
- **25%, 50%, 75%**: quartiles.

You can also pass arguments like `percentiles=[0.1, 0.9]` to see custom percentile values.

Lets see the summary statistics for our nfl dataframe:

In [ ]:
nfl_suspension_df_sorted.describe().round(2) # the last function .round() allows us to keep our values with as meny decimals we need

**Do you remember what graph we could create from this summary statistics?**

## Saving a Dataframe in our google drive

To save a dataframe into google drive after cleaning, managing and wrangling the data if nessesary:

1) Mount the google drive into google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

2) use `df.to_csv(file_path, index=False)` to save the final version of the dataframe into the google drive. Change the name of the file in the path to any desired name.


In [ ]:
# Define file path in Google Drive
file_path = '/content/drive/MyDrive/Lehman College Spring 2025/MAT 301/datasets/nfl_final_version.csv'

# Save DataFrame to CSV
nfl_suspension_df_sorted.to_csv(file_path, index=False)

print("File saved to Google Drive:", file_path)

## 4.  Measures of Central Tendency

**Packages Needed**  
- **NumPy**: A fundamental Python package for scientific computing, providing efficient arrays and mathematical operations.   
- **SciPy (`scipy.stats`)**: Builds on NumPy, offering advanced statistical tools. In particular, `trim_mean` helps us handle outliers by trimming data.





### 4.1 Mean
- The **mean** (arithmetic average) of (n) numbers $(x_1, x_2, \ldots, x_n)$ is:
  $
  \text{Mean} = \frac{x_1 + x_2 + \dots + x_n}{n}
  $
- **Limitation**: The mean is sensitive to outliers.
  
**Example in Python (using Numpy):**
```python
df['column_name'].mean()
```
Lets calculate the overall mean games for our nfl dataframe:

In [ ]:
games_mean = nfl_suspension_df_sorted['games'].mean().round(2)

print(f'the overall games mean is {games_mean}')

### 4.2 Median
- The **median** is the middle value when the data is in order.
  - If \(n\) (the number of data points) is **odd**, the median is the middle value.
  - If \(n\) is **even**, the median is the average of the two middle values.
- **Advantage**: The median is more robust to outliers.

**Example in Python (using Pandas):**
```python
df['column_name'].median()
```
Lets calculate the overall median games for our nfl dataframe:

In [ ]:
games_median = nfl_suspension_df_sorted['games'].median()

print(f'the overall games median is {games_median}')


### 4.3 Mode
- The **mode** is the most frequently occurring value in the data.
- There can be more than one mode if multiple values tie for most frequent.
- Especially useful for **categorical** or **discrete** data.

**Example in Python (using Pandas):**
```python
df['column_name'].mode()
```

Lets calculate the overall mode games for our nfl dataframe:


In [ ]:
games_mode = nfl_suspension_df_sorted['games'].mode()

print(f'the overall games mode is: \n ')

display(games_mode)

### 4.4 Trimmed Mean
- A **trimmed mean** removes a certain percentage of the smallest and largest values before computing the average, helping reduce the impact of outliers.

A simple approach:
1. Sort your values.
2. Remove a fixed fraction (e.g., 10%) of the data from each end.
3. Compute the mean of the remaining data.

**Example in Python (using `scipy.stats`):**
```python
from scipy.stats import trim_mean

trimmed = trim_mean(data_array, proportiontocut=0.1)  # Trim 10% from each tail
```

Lets calculate the 5% trimmed mean games for our nfl dataframe:

In [ ]:
from scipy.stats import trim_mean

games_trim_mean = trim_mean(nfl_suspension_df_sorted['games'], proportiontocut=0.05).round(2)

print(f'the overall games trim mean is {games_trim_mean}')

Lets observe all the descriptive statistics together:

In [ ]:
print("Descriptive Statistics for games column in the nfl suspension dataframe")

print(f"\nMean : {games_mean}\nMedian: {games_median}\nMode: {games_mode[0]}\nTrimmed Mean (5%): {games_trim_mean}")

## Discussion:

Which metric should we use, and what are the pros and cons of each option?

## 5. Summary and Key Takeaways
- **Data Wrangling**: Slicing, sorting, and grouping are fundamental to shaping your dataset to answer specific questions.
- **Central Tendency**: Mean, median, and mode each tell us different things about the "center" of data.
- **Outliers**: Trimmed mean can help reduce the effect of extreme outliers.
- **Choosing the Right Metric**: Always consider the data distribution and the presence of outliers when deciding which measure of central tendency to use.

